# Groupby, Pivot, Merge

In [1]:
import pandas as pd 
import numpy as np 

univ = ['AAPL','MSFT','BAC','GS']
dates = pd.date_range('20110101','20201231')
df = pd.DataFrame(np.random.randn(len(dates),len(univ)),index=dates,columns=univ)
df

,AAPL,MSFT,BAC,GS
2011-01-01,0.402659,-0.791877,-0.144330,1.843453
2011-01-02,1.290299,-0.336174,-1.914158,0.303239
2011-01-03,1.325322,0.207479,2.223407,1.469998
2011-01-04,0.265566,0.388181,-2.273731,0.415186
2011-01-05,1.415640,0.517858,1.470453,-1.011676
...,...,...,...,...
2020-12-27,-1.186698,0.613803,-0.641926,-2.792709
2020-12-28,-2.234191,0.521067,-1.265944,-0.785017
2020-12-29,1.228968,-0.432493,0.235560,0.682825
2020-12-30,-0.731760,0.787159,0.555993,0.559049


In [2]:
sector = {'AAPL':'Tech','MSFT':'Tech','BAC':'Fin','GS':'Fin'}
data=[]
for dt in dates:
    for x in univ:
        data.append([x,sector[x],dt,np.random.randn(),np.random.randn()])
data = pd.DataFrame(data,columns=['ticker','sector','date','signal1','signal2'])
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.082059,-2.940498
1,MSFT,Tech,2011-01-01,2.037516,-0.604571
2,BAC,Fin,2011-01-01,-0.277104,-0.044196
3,GS,Fin,2011-01-01,-0.129271,-0.086452
4,AAPL,Tech,2011-01-02,-1.687997,0.484285
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,0.310247,0.388901
14608,AAPL,Tech,2020-12-31,-0.087066,-1.390499
14609,MSFT,Tech,2020-12-31,-0.880014,-1.036360
14610,BAC,Fin,2020-12-31,-0.499025,-0.196924


## Groupby

In [3]:
# whats the average signal of each ticker?
data.groupby('ticker').mean()

,signal1,signal2
ticker,,
AAPL,0.009718,0.013495
BAC,0.012939,0.002670
GS,0.013280,-0.002070
MSFT,0.007759,-0.005093


In [4]:
# whats the average signal of each sector on each day?
data.groupby(['sector','date']).mean()

signal1   signal2
sector date                          
Fin    2011-01-01 -0.203187 -0.065324
       2011-01-02 -1.006247  1.047700
       2011-01-03  1.052026  0.588958
       2011-01-04 -0.448476 -0.334251
       2011-01-05 -1.251335  0.432628
...                     ...       ...
Tech   2020-12-27 -0.709643  0.275530
       2020-12-28  0.244018  0.534283
       2020-12-29  0.005680  0.482225
       2020-12-30 -0.742111  0.633299
       2020-12-31 -0.483540 -1.213430

[7306 rows x 2 columns]

In [5]:
# can index before applying the function
data.groupby(['sector','date'])[['signal1']].mean()

signal1
sector date                
Fin    2011-01-01 -0.203187
       2011-01-02 -1.006247
       2011-01-03  1.052026
       2011-01-04 -0.448476
       2011-01-05 -1.251335
...                     ...
Tech   2020-12-27 -0.709643
       2020-12-28  0.244018
       2020-12-29  0.005680
       2020-12-30 -0.742111
       2020-12-31 -0.483540

[7306 rows x 1 columns]

In [6]:
# use apply with groupby to pass an arbitrary function
def max_minus_min(x):
    return x.max()-x.min()

data.groupby(['sector','date'])[['signal1','signal2']].apply(max_minus_min)

signal1   signal2
sector date                          
Fin    2011-01-01  0.147833  0.042257
       2011-01-02  0.230941  2.066270
       2011-01-03  0.575121  1.563660
       2011-01-04  0.808406  1.274188
       2011-01-05  0.123482  0.053187
...                     ...       ...
Tech   2020-12-27  1.148023  0.013813
       2020-12-28  2.319763  3.665758
       2020-12-29  0.714555  1.234577
       2020-12-30  1.048836  1.587778
       2020-12-31  0.792948  0.354139

[7306 rows x 2 columns]

In [7]:
# return a dataframe instead of a series 
def demean(x):
    return x - x.mean()

data.groupby(['sector','date'])[['signal1','signal2']].apply(demean)

,signal1,signal2
0,-1.059787,-1.167963
1,1.059787,1.167963
2,-0.073917,0.021128
3,0.073917,-0.021128
4,-1.214564,1.143236
...,...,...
14607,0.398143,-0.594987
14608,0.396474,-0.177070
14609,-0.396474,0.177070
14610,-0.485247,0.828350


In [9]:
df = data.groupby(['sector','date'])[['signal1','signal2']].mean()
df

signal1   signal2
sector date                          
Fin    2011-01-01 -0.203187 -0.065324
       2011-01-02 -1.006247  1.047700
       2011-01-03  1.052026  0.588958
       2011-01-04 -0.448476 -0.334251
       2011-01-05 -1.251335  0.432628
...                     ...       ...
Tech   2020-12-27 -0.709643  0.275530
       2020-12-28  0.244018  0.534283
       2020-12-29  0.005680  0.482225
       2020-12-30 -0.742111  0.633299
       2020-12-31 -0.483540 -1.213430

[7306 rows x 2 columns]

In [10]:
df.groupby(level=0).mean()

,signal1,signal2
sector,,
Fin,0.013110,0.000300
Tech,0.008739,0.004201


In [11]:
month = [x.month for x in data['date']]

In [12]:
data.groupby(month).mean()

,signal1,signal2
1,-0.057283,0.053765
2,0.011552,-0.035065
3,0.040477,0.047933
4,0.031698,-0.037745
5,0.023019,0.042808
6,0.053481,0.001574
7,0.039846,-0.013846
8,-0.011966,-0.034617
9,0.003792,0.015620
10,0.000526,-0.010995


In [13]:
for key,val in data.groupby('sector'):
    print (key)

Fin
Tech


In [14]:
# iterating in a groupby
for key,val in data.groupby('sector'):
    print (val)

      ticker sector       date   signal1   signal2
2        BAC    Fin 2011-01-01 -0.277104 -0.044196
3         GS    Fin 2011-01-01 -0.129271 -0.086452
6        BAC    Fin 2011-01-02 -1.121717  0.014565
7         GS    Fin 2011-01-02 -0.890777  2.080835
10       BAC    Fin 2011-01-03  0.764465  1.370788
...      ...    ...        ...       ...       ...
14603     GS    Fin 2020-12-29 -0.512425 -0.784363
14606    BAC    Fin 2020-12-30 -0.486039  1.578875
14607     GS    Fin 2020-12-30  0.310247  0.388901
14610    BAC    Fin 2020-12-31 -0.499025 -0.196924
14611     GS    Fin 2020-12-31  0.471470 -1.853624

[7306 rows x 5 columns]
      ticker sector       date   signal1   signal2
0       AAPL   Tech 2011-01-01 -0.082059 -2.940498
1       MSFT   Tech 2011-01-01  2.037516 -0.604571
4       AAPL   Tech 2011-01-02 -1.687997  0.484285
5       MSFT   Tech 2011-01-02  0.741130 -1.802187
8       AAPL   Tech 2011-01-03  0.520508 -2.129498
...      ...    ...        ...       ...       ...
14601 

## Pivot

In [16]:
data

,ticker,sector,date,signal1,signal2
0,AAPL,Tech,2011-01-01,-0.082059,-2.940498
1,MSFT,Tech,2011-01-01,2.037516,-0.604571
2,BAC,Fin,2011-01-01,-0.277104,-0.044196
3,GS,Fin,2011-01-01,-0.129271,-0.086452
4,AAPL,Tech,2011-01-02,-1.687997,0.484285
...,...,...,...,...,...
14607,GS,Fin,2020-12-30,0.310247,0.388901
14608,AAPL,Tech,2020-12-31,-0.087066,-1.390499
14609,MSFT,Tech,2020-12-31,-0.880014,-1.036360
14610,BAC,Fin,2020-12-31,-0.499025,-0.196924


In [17]:
# put Signal1 into a df with columns tickers and rows dates
df1 = data.set_index(['date','ticker'])['signal1'].unstack(level=1)
df1

ticker,AAPL,BAC,GS,MSFT
date,,,,
2011-01-01,-0.082059,-0.277104,-0.129271,2.037516
2011-01-02,-1.687997,-1.121717,-0.890777,0.741130
2011-01-03,0.520508,0.764465,1.339587,1.637159
2011-01-04,1.290892,-0.852679,-0.044272,1.358168
2011-01-05,-0.562090,-1.189594,-1.313076,1.592343
...,...,...,...,...
2020-12-27,-1.283655,-0.207240,0.671878,-0.135632
2020-12-28,-0.915863,1.449401,-1.381277,1.403900
2020-12-29,-0.351598,-0.167941,-0.512425,0.362957


In [18]:
df2 = data.pivot_table(index='date',columns='ticker',values='signal1')
df2

ticker,AAPL,BAC,GS,MSFT
date,,,,
2011-01-01,-0.082059,-0.277104,-0.129271,2.037516
2011-01-02,-1.687997,-1.121717,-0.890777,0.741130
2011-01-03,0.520508,0.764465,1.339587,1.637159
2011-01-04,1.290892,-0.852679,-0.044272,1.358168
2011-01-05,-0.562090,-1.189594,-1.313076,1.592343
...,...,...,...,...
2020-12-27,-1.283655,-0.207240,0.671878,-0.135632
2020-12-28,-0.915863,1.449401,-1.381277,1.403900
2020-12-29,-0.351598,-0.167941,-0.512425,0.362957


In [19]:
df1.equals(df2)

True

In [20]:
data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.mean)

sector,Fin,Tech
date,,
2011-01-01,-0.203187,0.977729
2011-01-02,-1.006247,-0.473433
2011-01-03,1.052026,1.078834
2011-01-04,-0.448476,1.324530
2011-01-05,-1.251335,0.515126
...,...,...
2020-12-27,0.232319,-0.709643
2020-12-28,0.034062,0.244018
2020-12-29,-0.340183,0.005680


In [21]:
data.pivot_table(index='date',columns='sector',values='signal1',aggfunc=np.max)

sector,Fin,Tech
date,,
2011-01-01,-0.129271,2.037516
2011-01-02,-0.890777,0.741130
2011-01-03,1.339587,1.637159
2011-01-04,-0.044272,1.358168
2011-01-05,-1.189594,1.592343
...,...,...
2020-12-27,0.671878,-0.135632
2020-12-28,1.449401,1.403900
2020-12-29,-0.167941,0.362957


## Merge
Check out documentation to learn more: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [22]:
data2=[]
for dt in dates:
    for x in ['Tech','Fin']:
        data2.append([x,dt,np.random.randn(),np.random.randn()])
        
data2 = pd.DataFrame(data2,columns=['sector','date','signal3','signal4'])
data2

,sector,date,signal3,signal4
0,Tech,2011-01-01,-1.442788,0.340442
1,Fin,2011-01-01,-0.099906,1.155059
2,Tech,2011-01-02,0.003710,-0.317468
3,Fin,2011-01-02,-0.355778,0.545832
4,Tech,2011-01-03,0.039990,-0.358809
...,...,...,...,...
7301,Fin,2020-12-29,-0.831405,0.536993
7302,Tech,2020-12-30,-1.828334,0.115904
7303,Fin,2020-12-30,0.799164,0.281818
7304,Tech,2020-12-31,-1.112453,1.217002


In [23]:
data.merge(data2,left_on=['sector','date'],right_on=['sector','date'])

,ticker,sector,date,signal1,signal2,signal3,signal4
0,AAPL,Tech,2011-01-01,-0.082059,-2.940498,-1.442788,0.340442
1,MSFT,Tech,2011-01-01,2.037516,-0.604571,-1.442788,0.340442
2,BAC,Fin,2011-01-01,-0.277104,-0.044196,-0.099906,1.155059
3,GS,Fin,2011-01-01,-0.129271,-0.086452,-0.099906,1.155059
4,AAPL,Tech,2011-01-02,-1.687997,0.484285,0.003710,-0.317468
...,...,...,...,...,...,...,...
14607,GS,Fin,2020-12-30,0.310247,0.388901,0.799164,0.281818
14608,AAPL,Tech,2020-12-31,-0.087066,-1.390499,-1.112453,1.217002
14609,MSFT,Tech,2020-12-31,-0.880014,-1.036360,-1.112453,1.217002
14610,BAC,Fin,2020-12-31,-0.499025,-0.196924,1.788914,1.892547
